## Wind Velocity Animation

### Import Libraries

In [1]:
import os
import geemap
import xarray as xr
from ipyleaflet import basemaps
from ipyleaflet.velocity import Velocity

### Setting basemap from GEE

In [2]:
Map = geemap.Map(center=[-2, 113], zoom=3, interpolation='nearest')
Map.add_layer(basemaps.CartoDB.DarkMatter)
Map

Map(center=[-2, 113], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

### Open dataset

In [3]:
ds_wind=xr.open_dataset('wind_java_sea.nc',  chunks={'time':1})
ds_wind

<xarray.Dataset>
Dimensions:    (time: 168, longitude: 65, latitude: 41)
Coordinates:
  * time       (time) datetime64[ns] 2003-01-31 2003-02-28 ... 2016-12-31
  * longitude  (longitude) float32 101.0 101.2 101.5 101.8 ... 116.5 116.8 117.0
  * latitude   (latitude) float32 1.0 0.75 0.5 0.25 ... -8.25 -8.5 -8.75 -9.0
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>

### New data (wind_speed as velocity)

In [4]:
wind_speed=(ds_wind.u10**2 + ds_wind.v10**2)**0.5

### Combining data to ds_wind

In [5]:
ds_wind['wind_speed'] = wind_speed
ds_wind

<xarray.Dataset>
Dimensions:     (time: 168, longitude: 65, latitude: 41)
Coordinates:
  * time        (time) datetime64[ns] 2003-01-31 2003-02-28 ... 2016-12-31
  * longitude   (longitude) float32 101.0 101.2 101.5 ... 116.5 116.8 117.0
  * latitude    (latitude) float32 1.0 0.75 0.5 0.25 ... -8.25 -8.5 -8.75 -9.0
Data variables:
    u10         (time, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>
    v10         (time, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>
    wind_speed  (time, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>

### Slicing data

In [6]:
seasonal_wind = ds_wind.groupby('time.season').mean('time')
seasonal_wind

<xarray.Dataset>
Dimensions:     (longitude: 65, latitude: 41, season: 4)
Coordinates:
  * longitude   (longitude) float32 101.0 101.2 101.5 ... 116.5 116.8 117.0
  * latitude    (latitude) float32 1.0 0.75 0.5 0.25 ... -8.25 -8.5 -8.75 -9.0
  * season      (season) object 'DJF' 'JJA' 'MAM' 'SON'
Data variables:
    u10         (season, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>
    v10         (season, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>
    wind_speed  (season, latitude, longitude) float32 dask.array<chunksize=(1, 41, 65), meta=np.ndarray>

In [7]:
jja = seasonal_wind.sel(season='JJA')
jja

<xarray.Dataset>
Dimensions:     (longitude: 65, latitude: 41)
Coordinates:
  * longitude   (longitude) float32 101.0 101.2 101.5 ... 116.5 116.8 117.0
  * latitude    (latitude) float32 1.0 0.75 0.5 0.25 ... -8.25 -8.5 -8.75 -9.0
    season      <U3 'JJA'
Data variables:
    u10         (latitude, longitude) float32 dask.array<chunksize=(41, 65), meta=np.ndarray>
    v10         (latitude, longitude) float32 dask.array<chunksize=(41, 65), meta=np.ndarray>
    wind_speed  (latitude, longitude) float32 dask.array<chunksize=(41, 65), meta=np.ndarray>

### Display visualization to the basemap

In [9]:
display_options = {
    'velocityType': 'Global Wind',
    'displayPosition': 'bottomleft',
    'displayEmptyString': 'No wind data',
}
wind = Velocity(
    data=jja, #JJA from my own data (ds_wind)
    name='Velocity',
    zonal_speed='u10', #from ds_wind
    meridional_speed='v10', #from ds_wind
    latitude_dimension='latitude', #from ds_wind
    longitude_dimension='longitude', #from ds_wind
    velocity_scale=0.01,
    max_velocity=20,
    display_options=display_options,
)
Map.add_layer(wind)

Note : After we run this code, the animation of wind vilocity will showing on the basemap